In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# url of the table
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# read the url and get the html content
t = requests.get(url)
html_content = t.content
# beautifulsoup magic - the table is identified by it class
html_soup = BeautifulSoup(html_content, 'html.parser')
sov_tables = html_soup.find('table', class_="wikitable sortable")
#on to pandas dataframe
df = pd.read_html(str(sov_tables))
df = df[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [44]:
#remove lines where Borough in not assigned
df1 = df[df.Borough != 'Not assigned']
df1.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [45]:
#use comma as separator for multi neighborhood boroughs
df2 =df1 
df2.replace(to_replace=" / ", value=", ", regex=True,inplace=True)
df2.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [46]:
#check on Neighborhoods = NaN
df2[(df2['Neighborhood'] == "NaN")]

,Postal code,Borough,Neighborhood


In [47]:
df2.shape

(103, 3)